## Business Problem
Your company now sees all the big companies creating original video content and they want to get in on the fun. They have decided to create a new movie studio, but they don’t know anything about creating movies. You are charged with exploring what types of films are currently doing the best at the box office. You must then translate those findings into actionable insights that the head of your company's new movie studio can use to help decide what type of films to create.

## Data
The data used was obtained from the links below: 

* [Box Office Mojo](https://www.boxofficemojo.com/)
* [IMDB](https://www.imdb.com/)
* [Rotten Tomatoes](https://www.rottentomatoes.com/)
* [TheMovieDB](https://www.themoviedb.org/)
* [The Numbers](https://www.the-numbers.com/)

![movie data erd](movie_data_erd.jpeg)

### Goal
The goal is to investigate movie performance to identify patterns, genres, or attributes of successful films.

### Objectives:
1.	Analyze the Movie Data:
    * Identify characteristics of high-performing movies, in terms of genre, budget, runtime, release date, or lead actors/directors.
    * Look for trends in movie popularity and revenue growth over recent years.
2.	Understand Market Demand:
    * Examine the types of movies audiences prefer i.e. genre
    * Identify seasonal trends like during holiday releases, summer etc. to determine the best time for movie releases.
3.	Identify Key Success Factors:
    * Explore factors like movie rating, production budget, etc.
    * Evaluate the impact of sequels or franchise films on box office revenue.
4.	Translate Findings into Recommendations:
    * Provide specific, actionable insights for the company to make data-informed decisions on the types of movies to create.
    * Recommend strategies around genre, budget, target demographics, and release timing
